In [36]:
import pandas as pd
import json
import folium


# https://blog.naver.com/kcchang61/221350672356

# 구단위 지도 그리기
geo_path = './data/hankuk1_geo.json'
geo_str = json.load(open(geo_path, encoding='utf-8')) # 딕셔너리 객체로 리턴


# 서울 동별 인구수, 면적 정보
pop = pd.read_csv('./data/서울동별인구데이터1.csv')
pop.drop('Unnamed: 0',axis=1, inplace=True)


# geojson과 동별 인구수를 합친 엑셀 파일 불러오기 --> 구로구 항동 누락값은 직접 추가
final_data = pd.read_excel('./data/통합정리.xlsx', index_col='주소')  
final_data.columns = final_data.columns.map(str)

final_data.info()


# # 지역별, 동별 구분 경계선 
map = folium.Map(location=[37.5502, 126.982], zoom_start=13,
                 tiles='Stamen Terrain')

nm1 = 'adm_nm'
nm2 = '인구밀도'

print(final_data[nm].isnull())

fmap=folium.Choropleth(geo_data = geo_str,
                       data = final_data[nm1],
                       columns = [final_data[nm1], final_data[nm2]], 
                       fill_color='YlOrRd', fill_opacity=0.3, line_opacity=0.9,
                       threshold_scale=[10000, 20000, 30000, 40000, 50000], 
#                        key_on='feature.properties.adm_nm'
                      ).add_to(map)                             


# 지역 이름을 툴팁으로                                  
fmap.geojson.add_child(
    folium.features.GeoJsonTooltip(['adm_nm'],labels=False) # 해당 지역마다 툴팁으로 마우스오버 작동하게 만듦
)





# # 지하철 리스트를 데이터프레임 변환
df = pd.read_excel('./data/지하철노선위경도정보3.xlsx') # 데이터 프레임 형성

# 인근 지역 마커 카운터(마커 클러스터)
from folium.plugins import MarkerCluster
marker_cluster = MarkerCluster().add_to(map)
# 지하철 역 클러스터링
for i in range(len(df)):
    x = df.iloc[i][1]
    y = df.iloc[i][2]
    z = df.iloc[i][0]
    folium.Marker(
        location=[x,y],
        popup=z,
        icon=folium.Icon(color='blue',icon='ok'),
      ).add_to(marker_cluster)
    




# Q :보행접근성을 어떻게 평가할 것인가? 
# A :동별 면적을 구하고 내부 지하철역 개수로 그 면적을 나눈다. 즉 면적이 작을수록 보행접근성이 높다!
# How? : 동별 지하철 갯수를 세야 함



    
    
display(map)

<class 'pandas.core.frame.DataFrame'>
Index: 425 entries, 서울특별시 종로구 사직동 to 서울특별시 구로구 오류2동
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   OBJECTID  425 non-null    int64  
 1   adm_nm    425 non-null    object 
 2   adm_cd    425 non-null    int64  
 3   adm_cd2   425 non-null    int64  
 4   sidonm    425 non-null    object 
 5   sggnm     425 non-null    object 
 6   기간        425 non-null    int64  
 7   자치구       425 non-null    object 
 8   동         425 non-null    object 
 9   인구        425 non-null    object 
 10  면적        425 non-null    float64
 11  인구밀도      425 non-null    object 
dtypes: float64(1), int64(4), object(7)
memory usage: 43.2+ KB
주소
서울특별시 종로구 사직동     False
서울특별시 종로구 삼청동     False
서울특별시 종로구 부암동     False
서울특별시 종로구 평창동     False
서울특별시 종로구 무악동     False
                  ...  
서울특별시 강동구 둔촌2동    False
서울특별시 강동구 암사1동    False
서울특별시 강동구 천호2동    False
서울특별시 강동구 길동      False
서울특별시 구로구 오류2동    False
N

In [ ]:
# # 동별 면적 불러오기 --> 면적 단위는 km^2 // 면적 이미 있음
# area = pd.read_csv('./data/서울시행정구역동별통계.txt', sep="\t")
# # 자치구, 동에 "합계", "소계"가 포함된 행 없애기
# area_1 = area['자치구'].isin(['합계','소계'])
# area_2 = area[~area_1]
# area_3 = area_2['동'].isin(['합계','소계'])

# area_last = area_2[~area_3]
# # display(area_last.head())


# all_data = pd.merge(pop_last, area_last, how='left', on='동')
# # display(all_data)


# ====================================================================


# 동별 중앙값 불러오기
dong = pd.read_csv('./data/행정동별위치정보.csv', encoding='euc-kr')
dong = dong[['행정동명', '경도', '위도']]
dong.rename(columns = {'행정동명': '동'}, inplace=True)



# # # 동별 써클 그리기 --> 동별 중앙값을 찾은 뒤 거기서 반지름 50을 주자
# for dong,lat,lng in zip(all_data.동, all_data.위도, all_data.경도):
#     folium.CircleMarker(
#       [lat,lng],
#       radius=100,
#       color='#00ff00', # 테두리
#       fill_color='#ff000099', # 원 칼라 // 투명도도 같이 준다.
#       popup=dong
#     ).add_to(map)
#     folium.Marker(
#         [lat,lng],
#         icon=folium.Icon(color='red')
#     ).add_to(map)


## =============================================================


In [73]:
# geojson에서 특정 값 추출하기!!

import json

geo_path = './data/hankuk1_geo.json'
geo_str = json.load(open(geo_path, encoding='utf-8')) 

geo_nm = []
for i in range(425):
    points = geo_str['features'][i]['properties']
#     print(points)
    geo_nm.append(points)
geo_data = pd.DataFrame(geo_nm)
display(geo_data)

geo_loc = []
for i in range(425):
    points = geo_str['features'][i]['geometry']['coordinates']
    geo_loc.append(points)
    
geo_data1 =pd.DataFrame(geo_loc)


,OBJECTID,adm_nm,adm_cd,adm_cd2,sidonm,sggnm
0,0,서울특별시 종로구 사직동,1101053,1111053000,서울특별시,종로구
1,1,서울특별시 종로구 삼청동,1101054,1111054000,서울특별시,종로구
2,2,서울특별시 종로구 부암동,1101055,1111055000,서울특별시,종로구
3,3,서울특별시 종로구 평창동,1101056,1111056000,서울특별시,종로구
4,4,서울특별시 종로구 무악동,1101057,1111057000,서울특별시,종로구
...,...,...,...,...,...,...
420,420,서울특별시 강동구 둔촌2동,1125071,1174070000,서울특별시,강동구
421,421,서울특별시 강동구 암사1동,1125072,1174057000,서울특별시,강동구
422,422,서울특별시 강동구 천호2동,1125073,1174061000,서울특별시,강동구
423,423,서울특별시 강동구 길동,1125074,1174068500,서울특별시,강동구
